# Contributed packages worth knowing about

## Packages from [JuliaStats](https://github.com/JuliaStats/)

### The `Distributions` package

The `Distributions` package provides a comprehensive list of univariate, multivariate and matrix distributions and a set of generic functions that can be applied to them.  See the [documentation](http://distributionsjl.readthedocs.org/en/latest/) for details on the coverage.

Note that some methods refer to the distribution __type__ (e.g. `Normal`) instead of a distribution __instance__ (e.g. `Normal(0.,1.)`)

In [1]:
using Distributions
d = Normal()        # standard normal (Gaussian) distribution

Distributions.Normal(μ=0.0, σ=1.0)

In [2]:
x = rand(d,20)     # sample of size 20 from a standard normal

20-element Array{Float64,1}:
 -1.37163 
  0.108703
 -0.614373
  0.908453
 -0.616237
  1.30055 
  1.82147 
  1.56462 
  1.27457 
 -0.407733
 -0.691858
 -1.49719 
  0.850856
 -0.250366
  1.22701 
 -1.0103  
 -0.47893 
  0.512367
 -0.433555
  0.818587

In [3]:
de = fit_mle(Normal,x)   # maximum likelihood estimates of normal dist. pars.

Distributions.Normal(μ=0.15075083842683862, σ=0.9912678731753393)

In [4]:
mean(de)

0.15075083842683862

In [5]:
std(de)

0.9912678731753393

In [6]:
var(de)

0.9826119963895605

In [7]:
kurtosis(de)

0.0

In [8]:
skewness(de)

0.0

In [9]:
loglikelihood(de,x)

-28.203361159103117

In [10]:
ss = suffstats(Normal,x)

Distributions.NormalStats(3.0150167685367726,0.15075083842683862,19.65223992779121,20.0)

In [11]:
fieldnames(ss)'

1x4 Array{Symbol,2}:
 :s  :m  :s2  :tw

In [12]:
fit_mle(Normal,ss)   # can fit from sufficient statistics

Distributions.Normal(μ=0.15075083842683862, σ=0.9912678731753393)

It is also possible to do `map` (maximum a posteriori) estimation using a conjugate prior.  Again, see the documentation.

### The `DataArrays` package

This is a lightweight package to define types that can contain `NA`'s.  It was split off from the `DataFrames` package because loading the whole of `DataFrames` takes a while.  (This will change when pre-compiled packages are more easily constructed.)

The basic types defined in `DataArrays` are `NA`, used for literal NA input, the `DataArray`, like a numeric or integer vector in `R`, and the `PooledDataVector`, like a `factor` in `R`.

The concept of an `NA` is built into many of the `R` types.  In `Julia` a `DataArray` or `PooledDataArray` is composed of the data and a separate `bitarray` of indicators of missingness.

In [2]:
using DataArrays
v = @data([2,1,NA,4])

4-element DataArrays.DataArray{Int64,1}:
 2  
 1  
  NA
 4  

In [14]:
fieldnames(v)

2-element Array{Symbol,1}:
 :data
 :na  

In [15]:
typeof(v).types

svec(Array{Int64,1},BitArray{1})

In [16]:
v.data

4-element Array{Int64,1}:
 2
 1
 2
 4

In [17]:
v.na

4-element BitArray{1}:
 false
 false
  true
 false

In [18]:
isna(v)

4-element BitArray{1}:
 false
 false
  true
 false

In [19]:
dropna(v)

3-element Array{Int64,1}:
 2
 1
 4

The `PooledDataArray`, generated with the `@pdata` macro call, consists of a `pool` array (similar to the `levels` in an `R` `factor` object) and a unsigned integer vector `refs`.

In [20]:
d = Bernoulli()
sex = @pdata [rand(d) ≠ 0 ? 'F' : 'M' for i in 1:1000]

1000-element DataArrays.PooledDataArray{Char,UInt32,1}:
 'M'
 'F'
 'M'
 'M'
 'M'
 'F'
 'F'
 'F'
 'M'
 'F'
 'M'
 'F'
 'F'
 ⋮  
 'M'
 'F'
 'M'
 'F'
 'F'
 'F'
 'M'
 'F'
 'F'
 'F'
 'F'
 'M'

In [21]:
sex.pool

2-element Array{Char,1}:
 'F'
 'M'

In [22]:
sex.refs

1000-element Array{UInt32,1}:
 0x00000002
 0x00000001
 0x00000002
 0x00000002
 0x00000002
 0x00000001
 0x00000001
 0x00000001
 0x00000002
 0x00000001
 0x00000002
 0x00000001
 0x00000001
          ⋮
 0x00000002
 0x00000001
 0x00000002
 0x00000001
 0x00000001
 0x00000001
 0x00000002
 0x00000001
 0x00000001
 0x00000001
 0x00000001
 0x00000002

In [23]:
sex = compact(sex)  # provides a more compact representation, if possible

1000-element DataArrays.PooledDataArray{Char,UInt8,1}:
 'M'
 'F'
 'M'
 'M'
 'M'
 'F'
 'F'
 'F'
 'M'
 'F'
 'M'
 'F'
 'F'
 ⋮  
 'M'
 'F'
 'M'
 'F'
 'F'
 'F'
 'M'
 'F'
 'F'
 'F'
 'F'
 'M'

In [24]:
sex.refs

1000-element Array{UInt8,1}:
 0x02
 0x01
 0x02
 0x02
 0x02
 0x01
 0x01
 0x01
 0x02
 0x01
 0x02
 0x01
 0x01
    ⋮
 0x02
 0x01
 0x02
 0x01
 0x01
 0x01
 0x02
 0x01
 0x01
 0x01
 0x01
 0x02

A few other functions common to an `R` programmer, like `rep`, are in the `DataArrays` package.

In [3]:
whos(DataArrays)

                             /     38 KB     Function


                         @data   1533 bytes  Function
                        @pdata   1249 bytes  Function
             AbstractDataArray    188 bytes  DataType
            AbstractDataMatrix     80 bytes  TypeConstructor
            AbstractDataVector     80 bytes  TypeConstructor
             AbstractHistogram    228 bytes  DataType
                     CoefTable    284 bytes  DataType
                     DataArray    220 bytes  DataType
                    DataArrays    674 KB     Module
                    DataMatrix     80 bytes  TypeConstructor
                    DataVector     80 bytes  TypeConstructor
                    EachDropNA    168 bytes  DataType
                    EachFailNA    168 bytes  DataType
                 EachReplaceNA    220 bytes  DataType
                      FastPerm    284 bytes  DataType
                     Histogram    272 bytes  DataType
                        L1dist   1279 bytes  Function
                        L2dist    577 bytes  Function
  

### The `DataFrames` package

The `DataFrame` type and methods for working with it are defined in the `DataFrames` package.  There is online documentation but, at least in my browser, the formatting is horrible. I would recommend reading the PDF file instead.

The `DataFrames` package is where the formula language and types like `ModelFrame` and `ModelMatrix` are defined.  Many familiar examples of data frames are available in the `RDatasets` package.

In [26]:
using DataFrames, RDatasets
ds = dataset("lme4","Dyestuff")

,Batch,Yield
1,A,1545
2,A,1440
3,A,1440
4,A,1520
5,A,1580
6,B,1540
7,B,1555
8,B,1490
9,B,1560
10,B,1495


                         @data   1533 bytes  Function
                        @pdata   1545 bytes  Function
             AbstractDataArray    188 bytes  DataType
            AbstractDataMatrix     80 bytes  TypeConstructor
            AbstractDataVector     80 bytes  TypeConstructor
             AbstractHistogram    228 bytes  DataType
                     CoefTable    284 bytes  DataType
                     DataArray    220 bytes  DataType
                    DataArrays    803 KB     Module
                    DataMatrix     80 bytes  TypeConstructor
                    DataVector     80 bytes  TypeConstructor
                    EachDropNA    168 bytes  DataType
                    EachFailNA    168 bytes  DataType
                 EachReplaceNA    220 bytes  DataType
                      FastPerm    284 bytes  DataType
                     Histogram    272 bytes  DataType
                        L1dist   1279 bytes  Function
                        L2dist    577 bytes  Function
  

  likely near /home/bates/.julia/v0.4/RDatasets/src/dataset.jl:1
  likely near /home/bates/.julia/v0.4/RDatasets/src/dataset.jl:1
  likely near /home/bates/.julia/v0.4/RDatasets/src/datasets.jl:1


In [27]:
names(ds)

2-element Array{Symbol,1}:
 :Batch
 :Yield

Indivual columns can be accessed by name using symbols (e.g. `:Yield`).  This means that the column names should be valid Julia identifiers.  Among other things, they cannot contain the dot or period character (`.`).

In [28]:
ds[:Yield]

30-element DataArrays.DataArray{Int32,1}:
 1545
 1440
 1440
 1520
 1580
 1540
 1555
 1490
 1560
 1495
 1595
 1550
 1605
    ⋮
 1465
 1545
 1595
 1630
 1515
 1635
 1625
 1520
 1455
 1450
 1480
 1445

The `DataFrame` constructor can be given `<name>=<value>` pairs.

In [29]:
x = 1.:10.;
ϵ = rand(Normal(0.,0.1),length(x));
β = [4.2,1.1];
ytrue = [ones(length(x)) x]*β;
dd = DataFrame(x=x,ytrue = ytrue, y = ytrue + ϵ)

,x,ytrue,y
1,1.0,5.300000000000001,5.4266459954573465
2,2.0,6.4,6.547618585779938
3,3.0,7.5,7.4126925420952325
4,4.0,8.600000000000001,8.502478506094308
5,5.0,9.7,9.459142933970893
6,6.0,10.8,10.798058744556116
7,7.0,11.900000000000002,11.791803744236113
8,8.0,13.0,13.131877566162375
9,9.0,14.100000000000001,14.145992654678988
10,10.0,15.2,15.120811027815394


In `R` many modeling functions that use a formula/data representation first apply `model.frame` then `model.matrix`.  In the `DataFrames` package these are `ModelFrame` and `ModelMatrix`.  A `ModelFrame` is the reduction of the original `DataFrame` to only those columns that are used in the model and after application of the NA action.  It includes a `Terms` object, which describes the terms in the formula, again after some reduction and expansion.  Finally, a record is kept of which rows in the original data frame are represented in the derived frame.

In [30]:
mf = ModelFrame(y ~ x, dd)


DataFrames.ModelFrame(10x2 DataFrames.DataFrame
| Row | y       | x    |
|-----|---------|------|
| 1   | 5.42665 | 1.0  |
| 2   | 6.54762 | 2.0  |
| 3   | 7.41269 | 3.0  |
| 4   | 8.50248 | 4.0  |
| 5   | 9.45914 | 5.0  |
| 6   | 10.7981 | 6.0  |
| 7   | 11.7918 | 7.0  |
| 8   | 13.1319 | 8.0  |
| 9   | 14.146  | 9.0  |
| 10  | 15.1208 | 10.0 |,DataFrames.Terms(Any[:x],Any[:y,:x],2x2 Array{Int8,2}:
 1  0
 0  1,[1,1],true,true),Bool[true,true,true,true,true,true,true,true,true,true])

The `ModelMatrix` is constructed from the `ModelFrame`.

In [31]:
mm = ModelMatrix(mf)

DataFrames.ModelMatrix{Float64}(10x2 Array{Float64,2}:
 1.0   1.0
 1.0   2.0
 1.0   3.0
 1.0   4.0
 1.0   5.0
 1.0   6.0
 1.0   7.0
 1.0   8.0
 1.0   9.0
 1.0  10.0,[0,1])

The `assign` vector in this object maps columns to terms.  It is used when performing hypothesis tests, like `anova`.  At present the `model_response` function returns the value of the expression on the left-hand side of the formula.

In [32]:
model_response(mf)

10-element Array{Float64,1}:
  5.42665
  6.54762
  7.41269
  8.50248
  9.45914
 10.7981 
 11.7918 
 13.1319 
 14.146  
 15.1208 

These facilities are not developed as fully as those in `R`.

### The GLM Package

The `GLM` package provides functions to fit and analyse the linear models and generalized linear models.

In [33]:
using GLM
fm = lm(y ~ x, dd)

DataFrames.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredQR{Float64}},Float64}:

Coefficients:
             Estimate Std.Error t value Pr(>|t|)
(Intercept)   4.22575 0.0913301  46.269   <1e-10
x             1.09236 0.0147192 74.2132   <1e-11


In [34]:
fm = fit(LinearModel,y ~ x,dd)

DataFrames.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredQR{Float64}},Float64}:

Coefficients:
             Estimate Std.Error t value Pr(>|t|)
(Intercept)   4.22575 0.0913301  46.269   <1e-10
x             1.09236 0.0147192 74.2132   <1e-11


### StatsBase

The `StatsBase` package contains functions for sample statistics and many utilities.  There is online documentation.  Much of the design and implementation is by Dahua Lin who is a stickler for extracting every last ounce of performance.

### MLBase

The `MLBase` package contains many functions for data manipulation and reduction.  It uses the Machine Learning (ML) terminology.

## [JuliaOpt](http://www.juliaopt.org) packages

One of the areas in which Julia shines is optimization packages.  I mostly do nonlinear optimization subject to box constraints and use the `NLopt` package.  Many other types of optimization problems can be addressed with the `JuMP` package.

A recent addition is the [JuliaDiff](http://www.juliadiff.org) organization that provides several types of automatic differentiation packages for Julia.